In [ ]:
import urllib.request
import json
import csv
# import time
# import pandas as pd
# import uuid

##GoogleCloud API key
key = 'key=xxxxx'

##Functions
# to read google maps api routes. 
# Returns unfiltered response from API
# mode param 1 for driving, 2 - for transit

def routeQuery (origin, destination, mode):
    p1 = "https://maps.googleapis.com/maps/api/directions/json?" #original url
    p2 = "&origin="+str(origin) #point of origin
    p3 = "&destination="+str(destination) #point of destination
    p4 = "&"+key #API key
    p6 = "&units=metric&language=ru" #country parametres
    if mode == 1:  #selecting mode of transportation
        p5="&mode=driving"
    else: 
        p5="&mode=transit" ##Добавить сюда вычиление следующего буднего дня!!
    urlc=p1+p2+p3+p4+p5+p6
    raw=urllib.request.urlopen(urlc)
    decoded=json.load(raw)
    return decoded

## filter data output to get start/and points adress
def routePoints (parsed_jsn):
    data = [
    parsed_jsn['routes'][0]['legs'][0]['start_address'],
    parsed_jsn['routes'][0]['legs'][0]['end_address']
    ]
    return data
## filter data to get time (in sec) and dustance (in meters)
def routeParams (parsed_jsn):
    data = [
    parsed_jsn['routes'][0]['legs'][0]['duration']['value'],
    parsed_jsn['routes'][0]['legs'][0]['distance']['value']
    ]
    return data


In [ ]:

try:
    with open('routes.csv', 'r') as rf:
        rreader = csv.reader(rf)
        rlist = list(rreader)
    rlist.pop(0) #delet headers just in case
    print('Всего будет запрошено', len(rlist), "адресов")
except:
    print('Какая-то проблема с файлом адресов')


In [ ]:
result = []
n=0
for i in rlist:
    n += 1
    try:
        b = routeQuery(i[1],i[2],1)
        
        result.append([
            [i[0]],
            routePoints(b),
            routeParams(b),
            routeParams(routeQuery(i[1],i[2],2))
            ])
        print('Запрашиваем адрес',n,"из",len(rlist))
    except:
        print('Ошибка при работе с точкой',i[0])
#flattent results
flatlist=[]
for i in result:
    flatsub=[]
    
    for sub in i:
        for subsub in sub:
            flatsub.append(subsub)
    flatlist.append(flatsub)

print('Получен реузльтат для',len(flatlist), 'точек')


In [ ]:

with open('result.csv', 'w', newline='') as myfile:
     myfile.write('"routename","from","to","drive_dur","drive_len","transit_dur","transit_len"')
     myfile.write("\n")
     wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
     for i in flatlist:
        wr.writerow(i)

